In [2]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
import numpy as np
import os
import sys
import time
import datetime
import glob
sys.path.append('../py/')
from s027_kfold_ods import ods_kfold
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import logger_func, get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature, impute_feature
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()


2019-02-19 00:19:19,134 utils 400 [INFO]    [logger_func] start 


In [5]:
#========================================================================
# Args
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term', 'no_out_flg']
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
#========================================================================

#========================================================================
# Data Load
win_path = f'../features/4_winner/*.gz'
model_path_list = [f'../model/LB3670_70leaves_colsam0322/*.gz', '../model/E2_lift_set/*.gz', '../model/E3_PCA_set/*.gz', '../model/E4_mix_set/*.gz']
model_path = model_path_list[0]
win_path_list = glob.glob(model_path) + glob.glob(win_path)

base = utils.read_pkl_gzip('../input/base_no_out_clf.gz')[[key, target]]
# base_train = base[~base[target].isnull()].reset_index(drop=True)
# base_test = base[base[target].isnull()].reset_index(drop=True)

feature_list = utils.parallel_load_data(path_list=win_path_list)
df_feat = pd.concat(feature_list, axis=1)
df_feat = pd.concat([base, df_feat], axis=1)
#========================================================================

In [8]:
df_feat.to_csv('../output/0219_go_elo_dataset_LB3669.csv', index=False)

In [9]:
base = utils.read_pkl_gzip('../input/base_no_out_clf.gz')[[key, target, 'no_out_flg', 'clf_pred']]
base.to_csv('../output/0219_go_elo_classifier_pred_NoOutlierFlg.csv', index=False)

In [10]:
df_feat.head()

,card_id,target,190_fe1_all_auth1_term@,206_pst_auth1_cat1_lag02_diff_date_from_hist_min_month_mean@,114_mer_merchant_id_M_ID_86be58d7e0@,152_ah0__auth0_hist_card_id_size@,114_mer_merchant_id_M_ID_1f4773aa76@,556_lif_hist_regist_term_lift_206_pst_auth1_cat1_month_diff_mean@,556_lif_hist_regist_term_lift_203_pst_auth1_lag02_month_diff_mean@,212_rgt_lift_regist_term_new_cat0_purchase_amount_sum@,...,242_rad_freq_2Mover_auth1_all_flag-2_-4-plag-5_-10_purchase_amount_new_sum_per_installments_sum_mean@,241_rad_auth1_all_flag-2_-4-plag-5_-10_purchase_amount_new_sum_mean@,247_rad_up_cnt_ratio_lag0_-2_ratio_lag1_installments_max@,241_rad_auth1_all_flag0_-2-plag-3_-5_purchase_amount_new_sum_per_installments_sum_max@,241_rad_auth1_all_flag0_0-plag-1_-6_installments_sum_mean@,245_rad_min_Mlag-6_auth1_all_top-M1_merchant_flag-2_-4-plag-5_-7_purchase_amount_new_sum@,241_rad_auth1_all_flag0_0-plag-1_-6_purchase_amount_new_sum_per_installments_sum_max@,247_rad_up_cnt_ratio_lag0_-2_ratio_lag1_purchase_amount_new_sum_per_installments_sum@,241_rad_auth1_all_flag-1_-3-plag-4_-6_purchase_amount_new_sum_max@,245_rad_min_Mlag-6_auth1_all_top-M2_merchant_flag0_-1-plag-2_-3_purchase_amount_new_sum_per_installments_sum@
0,C_ID_92a2005557,-0.820283,242.0,NaN,0,13.0,0,-1.000000,0.894344,3.664062,...,1.636355,1.742212,-1.0,24.975000,0.216890,1.703125,11.040451,0.200000,49.950001,0.829306
1,C_ID_3d0044924f,0.392913,390.0,-354.799988,0,11.0,1,1.008662,0.987283,0.125488,...,1.312074,1.582895,0.0,7.610700,0.978889,0.218096,4.070000,0.714286,38.053501,18.412666
2,C_ID_d639edf6cd,0.688056,412.0,NaN,0,2.0,0,-1.000000,0.908511,0.056000,...,0.905455,1.575460,-1.0,0.871400,0.000000,1.288221,0.000000,-1.000000,0.871400,-1.000000
3,C_ID_186d6a6901,0.142495,154.0,-148.285721,0,NaN,0,0.918954,0.907294,0.425781,...,1.614608,4.321847,1.0,8.392619,0.478261,3.713333,8.392619,0.500000,4.863728,0.000000
4,C_ID_cdbd2c0db2,-0.159749,108.0,-56.099998,0,5.0,0,0.931988,0.898722,4.718750,...,0.000000,0.000000,-1.0,1.602850,0.698765,0.000000,4.619877,0.571429,0.000000,1.040527
